In [2]:
from IPython.core.debugger import set_trace
import random, numpy, sys, math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as mpl

Read experiment data from .csv files, filter and group according to needs

In [3]:
df = pd.read_csv('exp_gsemo_seed.csv')
df['algo'] = 'gsemo'

df_semo = pd.read_csv('exp_semo_seed.csv')
df_semo['algo'] = 'semo'

df_sms = pd.read_csv('exp_smsemoa.csv')
df_sms['algo'] = 'sms'

df = pd.concat([df, df_semo, df_sms])


df['puffer'] = df['NGEN']-df['gen']
df = df[df['puffer']>1]

df = df[df['M'] == 2]
df = df[df['seed'] <= 30]
df = df[((df['benchmark'] == 'COCZ') & (df['blockSize'].isin([3,5,7,10,12,15]))) |
                ((df['benchmark'] != 'COCZ') & (df['blockSize'].isin([5,10,15,20,25,40])))]

df['n'] = df['blockSize']*df['M']
df.loc[df['benchmark'] == 'COCZ', ['n']] *= 2

keys = ['algo','benchmark', 'M','blockSize','k','n', 'cxpb']

gdf = df.groupby(keys).agg(
    count=pd.NamedAgg(column="gen", aggfunc="count"),
    meanGen=pd.NamedAgg(column="gen", aggfunc="mean"),
    stdGen=pd.NamedAgg(column="gen", aggfunc="std"),
    varGen=pd.NamedAgg(column="gen", aggfunc="var"))

gdf = gdf.reset_index()
gdf['relStd'] = gdf['stdGen'] / gdf['meanGen']
gdf = gdf.sort_values(by=['algo','benchmark','M','blockSize','cxpb'])

Identify maximum (relative) standard deviation

In [4]:
print(gdf['stdGen'].max())
print(gdf['relStd'].max())

1522864.19885841
0.655419808169888


Display results for the NSGA-II

In [6]:
df_nsga = pd.read_csv('exp_nsga2_mu_is_fourTimes_front.csv')
display(df_nsga)

,benchmark,M,blockSize,k,cxpb,gen,NGEN,max_cover,cover_hist
0,LOTZ,2,10,0,0.0,999999,1000000,107,"[62, 84, 92, 92, 89, 80, 96, 90, 88, 86, 84, 9..."
1,OMM,2,10,0,0.0,999999,1000000,106,"[65, 91, 92, 89, 90, 85, 91, 93, 79, 88, 85, 8..."
2,LOTZ,2,10,0,0.0,999999,1000000,106,"[65, 91, 92, 89, 90, 85, 91, 93, 79, 88, 85, 8..."
3,OMM,2,10,0,0.0,999999,1000000,107,"[62, 84, 92, 92, 89, 80, 96, 90, 88, 86, 84, 9..."
4,LOTZ,2,10,0,0.0,999999,1000000,106,"[60, 91, 87, 85, 88, 90, 90, 89, 89, 95, 91, 9..."
5,OMM,2,10,0,0.0,999999,1000000,106,"[60, 91, 87, 85, 88, 90, 90, 89, 89, 95, 91, 9..."
6,COCZ,2,10,0,0.0,999999,1000000,107,"[62, 84, 92, 92, 89, 80, 96, 90, 88, 86, 84, 9..."
7,COCZ,2,10,0,0.0,999999,1000000,106,"[65, 91, 92, 89, 90, 85, 91, 93, 79, 88, 85, 8..."
8,COCZ,2,10,0,0.0,999999,1000000,106,"[60, 91, 87, 85, 88, 90, 90, 89, 89, 95, 91, 9..."
9,OJZJ,2,10,2,0.0,999999,1000000,102,"[57, 85, 89, 85, 78, 79, 82, 82, 86, 85, 87, 9..."


Identify maximum number of iterations it took any tested algorithm except NSGA-II to solve any benchmark on bitstrings with length n = 20

In [10]:
print(df[(df['blockSize']==10) & (df['M'] == 2) & (df['k']<=2)]['gen'].max())
print(df[(df['blockSize']==5) & (df['benchmark'] == 'COCZ') & (df['M'] == 2) & (df['k']<=2)]['gen'].max())


198587
6532


Configure matplotlib to our needs

In [11]:
from matplotlib.ticker import NullLocator
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'font.size': 28})
plt.rcParams['ytick.right'] = plt.rcParams['ytick.left'] = True
plt.rcParams['xtick.top'] = plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.labelright'] = False 
plt.rcParams['ytick.labelleft'] = True
plt.rcParams['xtick.labelbottom'] = True

Define method for displaying/saving a graphic showing the number of iterations it took the different algorithms to solve a specific setting/benchmark

In [31]:
def compareRuntimes(dfp, name, varx="n", varxname="n", log=False, save=False, ojzj=False):
    
    colors = {'gsemo': 'blue', 'semo':'green', 'sms':'red'}
    algos = {'gsemo', 'semo', 'sms'}
    
    fig, ax = plt.subplots()
    if log:
        ax.loglog()
    
    for algo in algos:
        if ojzj and algo == "semo": 
            continue
        df_tmp = dfp[dfp['algo']==algo]
        ax.plot(df_tmp[varx], df_tmp['meanGen'], 'x-', linewidth=2.5, color=colors[algo], label="mean")


    labels = [10,20,30,40] if ojzj else df_tmp[varx] 

    ax.xaxis.set_ticks(labels)
    ax.xaxis.set_ticklabels(labels)
    
    ax.xaxis.set_minor_locator(NullLocator())
    ax.xaxis.set_minor_formatter('')
    
    ax.yaxis.set_minor_locator(NullLocator())
    
    
    if save == 'pdf': 
        plt.savefig('images/{}.pdf'.format(name), bbox_inches = "tight")
        plt.clf()
        print('saved as pdf')
        
    else:
        plt.tight_layout()
        plt.title(name)
        plt.show()

Generate graphics for COCZ benchmark using above method

In [ ]:
save = 'pdf'
M = 2
df_tmp = gdf[(gdf['benchmark']=='COCZ') & (gdf['M']==M)]
bench = 'COCZ'
df_tmp = df_tmp[df_tmp['M']==M]
df_tmp = df_tmp[df_tmp['k']==0]
df_tmp = df_tmp[df_tmp['n'].isin([12,20,28,40,48,60])]
#display(df_tmp)
compareRuntimes(df_tmp, "{} with M={}".format(bench,M), save=save)
compareRuntimes(df_tmp, "{} with M={} log".format(bench,M), log=True, save=save)

Generate graphics for OMM, LOTZ, and OJZJ benchmark using above method

In [34]:
df_tmp = gdf
save = 'pdf'
for bench in ["OMM","LOTZ"]:
    for M in [2]:
        bound = lambda n : bounds[bench](n,M)
        compareRuntimes(df_tmp[(df_tmp['benchmark']==bench) & (df_tmp['M']==M)], "{} with M={}".format(bench,M), save=save)
        compareRuntimes(df_tmp[(df_tmp['benchmark']==bench) & (df_tmp['M']==M)], "{} with M={} log".format(bench,M), log=True,save=save)

for M in [2]:
    for k in [2]:
        bound = lambda n : bounds['OJZJ'](n,M,k)
        compareRuntimes(df_tmp[(df_tmp['benchmark']=='OJZJ') & (df_tmp['M']==M) & (df_tmp['k']==k)& (df_tmp['blockSize']<=20)],
                         "OJZJ with M={}, and k={}".format(M,k), save=save, ojzj=True)
        compareRuntimes(df_tmp[(df_tmp['benchmark']=='OJZJ') & (df_tmp['M']==M) & (df_tmp['k']==k)& (df_tmp['blockSize']<=20)],
                         "OJZJ with M={}, and k={} log".format(M,k), log=True, save=save, ojzj=True)

saved as pdf
saved as pdf
saved as pdf
saved as pdf
saved as pdf
saved as pdf
